In [1]:
import re
import os
import math
import torch
import random
import pandas as pd
import pytorch_lightning as pl

from tqdm import tqdm
from rouge import Rouge
from datetime import datetime

from torch import nn
from konlpy.tag import Mecab
from tensorboardX import SummaryWriter
from transformers import AutoTokenizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_id = "psyche/KoT5-summarization" ## "eenzeenee/t5-base-korean-summarization"
train_df = pd.read_csv('../dataset/cleaned_train.csv')
val_df = pd.read_csv('../dataset/cleaned_dev.csv')
test_df = pd.read_csv("../dataset/test.csv")

# new_df = pd.read_csv("../dataset/new_data.csv").sample(10000)
# train_df = pd.concat([train_df, new_df])
# print(train_df.shape)

In [3]:
epochs = 1000
batch_size = 4
accumulation_steps = 4
num_workers = 0
patience = 10

init_lr = 0.000001
max_lr = 0.00001
weight_decay = 0
warmup_epochs = 10
T_0 = 100
T_mult = 1
T_gamma = 0.5

dig_max_len = 512
sum_max_len = 256

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=init_lr, weight_decay=weight_decay)

extra_tokens = [f"<extra_id_{i}>" for i in range(500)]
remove_tokens = ['<usr>', f"{tokenizer.unk_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"] + extra_tokens
special_tokens = [
    '#Person1#',
    '#Person2#',
    '#Person3#',
    '#Person4#',
    '#Person5#',
    '#Person6#',
    '#Person7#',
    '#SSN#',
    '#Email#',
    '#Address#',
    '#Reaction#',
    '#CarNumber#',
    '#Movietitle#',
    '#DateOfBirth#',
    '#CardNumber#',
    '#PhoneNumber#',
    '#PassportNumber#',
    '<sep>'
]

tokenizer.add_special_tokens({"additional_special_tokens": special_tokens + extra_tokens})
print(tokenizer.special_tokens_map)

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#SSN#', '#Email#', '#Address#', '#Reaction#', '#CarNumber#', '#Movietitle#', '#DateOfBirth#', '#CardNumber#', '#PhoneNumber#', '#PassportNumber#', '<sep>', '<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<e

In [5]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, input_len, summ_len, is_train=True):
        self.tokenizer = tokenizer
        self.df = df.copy()
        self.source_len = input_len
        self.summ_len = summ_len
        self.is_train = is_train

        # 화자가 바뀔 때 SEP 토큰을 추가
        self.df.loc[:, 'dialogue'] = self.df['dialogue'].apply(self.add_sep_tokens)

        if self.is_train:
            # self.df['dialogue'] = self.df['dialogue'].apply(self.apply_augmentations)
            
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), 
                                       return_tensors="pt", 
                                       padding=True,
                                       add_special_tokens=True, 
                                       truncation=True, 
                                       max_length=dig_max_len, 
                                       return_token_type_ids=False).input_ids
            
            self.labels = tokenizer(self.df['summary'].tolist(), 
                                    return_tensors="pt", 
                                    padding=True,
                                    add_special_tokens=True, 
                                    truncation=True, 
                                    max_length=sum_max_len, 
                                    return_token_type_ids=False).input_ids
        else:
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), 
                                       return_tensors="pt", 
                                       padding=True,
                                       add_special_tokens=True, 
                                       truncation=True, 
                                       max_length=dig_max_len, 
                                       return_token_type_ids=False).input_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.is_train:
            return self.input_ids[idx], self.labels[idx]
        else:
            return self.input_ids[idx]


    def add_sep_tokens(self, dialogue):
        # 화자가 바뀔 때 SEP 토큰을 추가
        pattern = r'(#Person\d+#)'  # 화자를 나타내는 패턴
        parts = re.split(pattern, dialogue)  # 화자를 기준으로 대화 분리
        result = []
        prev_speaker = None
        for part in parts:
            if re.match(pattern, part):  # 화자가 바뀌면
                if prev_speaker and prev_speaker != part:
                    result.append('<sep>')  # SEP 토큰 추가
                prev_speaker = part
            result.append(part)
        return ''.join(result)

    ###
    def apply_augmentations(self, text):
        # 세 가지 변환 적용: text infilling, sentence permutation, token masking
        text = self.text_infilling(text)
        # text = self.sentence_permutation(text)
        text = self.token_masking(text)
        return text

    def text_infilling(self, text, fill_prob=0.15):
        # 15% 확률로 텍스트 일부를 빈칸 처리 (T5 스타일로 <extra_id_0>, <extra_id_1> 사용)
        words = text.split()
        num_to_fill = int(len(words) * fill_prob)
        fill_indices = random.sample(range(len(words)), num_to_fill)
        
        # <extra_id_0>, <extra_id_1> 등의 토큰을 사용해 infilling 처리
        extra_id = 0
        for idx in fill_indices:
            words[idx] = f'<extra_id_{extra_id}>'
            extra_id += 1  # 각 빈칸에 다른 extra_id 부여
        
        return ' '.join(words)

    def sentence_permutation(self, text):
        # 문장의 순서를 랜덤으로 섞기
        sentences = re.split(r'(?<=[.!?]) +', text)
        random.shuffle(sentences)
        return ' '.join(sentences)

    def token_masking(self, text, mask_prob=0.15):
        # 15% 확률로 토큰을 마스킹 (T5 스타일로 <extra_id_*> 사용)
        tokens = text.split()
        num_to_mask = int(len(tokens) * mask_prob)
        mask_indices = random.sample(range(len(tokens)), num_to_mask)
        
        # <extra_id_*> 형식으로 마스킹 처리
        extra_id = 0
        for idx in mask_indices:
            tokens[idx] = f'<extra_id_{extra_id}>'
            extra_id += 1
        
        return ' '.join(tokens)

In [6]:
train_dataset = CustomDataset(train_df[['dialogue', 'summary']], tokenizer, dig_max_len, sum_max_len)
val_dataset = CustomDataset(val_df[['dialogue', 'summary']], tokenizer, dig_max_len, sum_max_len)
test_dataset = CustomDataset(test_df[['dialogue']], tokenizer, dig_max_len, sum_max_len, is_train=False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [7]:
sample_dig = train_df.iloc[0]['dialogue']
sample_sum = train_df.iloc[0]['summary']

dig_tk = tokenizer(sample_dig)
sum_tk = tokenizer(sample_sum)

print(dig_tk)
print(sum_tk)

print(tokenizer.convert_ids_to_tokens(dig_tk.input_ids))
print(tokenizer.convert_ids_to_tokens(sum_tk.input_ids))

{'input_ids': [32100, 7074, 25907, 104, 14307, 26164, 25892, 1439, 284, 26526, 25932, 3130, 181, 25892, 290, 655, 36, 26510, 2952, 26087, 25889, 32101, 899, 26219, 2366, 2014, 198, 4045, 15, 8227, 25949, 25892, 25889, 32100, 18639, 25907, 7017, 114, 25984, 586, 899, 26219, 2366, 3463, 5653, 25892, 2868, 7064, 1110, 25892, 25889, 32101, 1094, 273, 25892, 318, 635, 1343, 10892, 655, 7059, 21131, 3580, 325, 25949, 26087, 25889, 32100, 4199, 4750, 25895, 17508, 343, 587, 7002, 748, 16314, 1112, 32, 1909, 25892, 6829, 6993, 7867, 172, 9361, 2868, 16, 20498, 25514, 25892, 25889, 32101, 11060, 25892, 25889, 32100, 949, 13664, 25892, 6993, 14855, 629, 25893, 17009, 2974, 25892, 13465, 6458, 3788, 26481, 865, 25892, 104, 14307, 26164, 25907, 5352, 24604, 25911, 2952, 26087, 25889, 32101, 216, 25892, 25889, 32100, 1391, 25906, 131, 12374, 25907, 5352, 25893, 19605, 25938, 5576, 12870, 1174, 4849, 181, 25892, 8477, 1581, 25919, 3984, 1110, 25892, 25889, 32101, 5683, 315, 2542, 1082, 25907, 22180,

In [8]:
sample_batch = train_dataset[0]
print(sample_batch[0].shape)

print(tokenizer.convert_ids_to_tokens(sample_batch[0]))

torch.Size([512])
['#Person1#', '▁안녕하세요', ',', '▁스', '미스', '씨', '.', '▁저는', '▁호', '킨', '스', '▁의사', '입니다', '.', '▁오늘', '▁왜', '▁오', '셨', '나요', '?', '▁', '<sep>', '#Person2#', '▁건강', '검', '진을', '▁받는', '▁것이', '▁좋을', '▁것', '▁같아서', '요', '.', '▁', '<sep>', '#Person1#', '▁그렇군요', ',', '▁당신은', '▁5', '년', '▁동안', '▁건강', '검', '진을', '▁받지', '▁않았습니다', '.', '▁매년', '▁받아야', '▁합니다', '.', '▁', '<sep>', '#Person2#', '▁알고', '▁있습니다', '.', '▁하지만', '▁아무', '▁문제가', '▁없다면', '▁왜', '▁의사를', '▁만나러', '▁가야', '▁하나', '요', '?', '▁', '<sep>', '#Person1#', '▁심각한', '▁질병', '을', '▁피하는', '▁가장', '▁좋은', '▁방법은', '▁이를', '▁조기에', '▁발견', '하는', '▁것입니다', '.', '▁그러니', '▁당신의', '▁건강을', '▁위해', '▁최소한', '▁매년', '▁한', '▁번은', '▁오세요', '.', '▁', '<sep>', '#Person2#', '▁알겠습니다', '.', '▁', '<sep>', '#Person1#', '▁여기', '▁보세요', '.', '▁당신의', '▁눈과', '▁귀', '는', '▁괜찮아', '▁보입니다', '.', '▁깊게', '▁숨을', '▁들이', '쉬', '세요', '.', '▁스', '미스', '씨', ',', '▁담배', '▁피우', '시', '나요', '?', '▁', '<sep>', '#Person2#', '▁네', '.', '▁', '<sep>', '#Person1#', '▁당신', '도', '▁알', '다시피

In [9]:
def ids_to_words(tokenizer, preds, labels):
    decoded_preds = tokenizer.batch_decode(preds, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()

    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]
        
    return replaced_predictions, replaced_labels

In [10]:
def compute_metrics(replaced_predictions, replaced_labels):
    rouge = Rouge()

    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)
    result = {key: value["f"] for key, value in results.items()}
    
    return result

In [11]:
class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [12]:
# def train(epoch, model, device, train_loader, optimizer, writer):
#     model.train()
#     total_loss = 0.0

#     for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Training Epoch {epoch}", leave=False):
#         input_ids = batch[0].to(device, dtype=torch.long)
#         labels = batch[1].to(device, dtype=torch.long)

#         optimizer.zero_grad()
#         outputs = model(input_ids=input_ids, labels=labels)
#         ce_loss = outputs.loss

#         ce_loss.backward()
#         optimizer.step()

#         total_loss += ce_loss.item()

#     avg_loss = total_loss / len(train_loader)

#     writer.add_scalar('Loss/train', avg_loss, epoch)
#     return avg_loss

def train(epoch, model, device, train_loader, optimizer, writer, accumulation_steps):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()

    for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Training Epoch {epoch}", leave=False):
        input_ids = batch[0].to(device, dtype=torch.long)
        labels = batch[1].to(device, dtype=torch.long)

        outputs = model(input_ids=input_ids, labels=labels)
        ce_loss = outputs.loss
        ce_loss = ce_loss / accumulation_steps 
        ce_loss.backward()

        if (idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += ce_loss.item() * accumulation_steps

    avg_loss = total_loss / len(train_loader)
    writer.add_scalar('Loss/train', avg_loss, epoch)
    return avg_loss

In [13]:
def validate(tokenizer, model, device, val_loader, writer, epoch):
    model.eval()
    total_loss = 0
    all_results = []
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for idx, batch in tqdm(enumerate(val_loader), total=len(val_loader), desc="Validating", leave=False):
            input_ids = batch[0].to(device, dtype=torch.long)
            labels = batch[1].to(device, dtype=torch.long)

            pred_ids = model.generate(input_ids=input_ids, max_length=sum_max_len, num_beams=4, repetition_penalty=2.0, 
                                      length_penalty=1.0, early_stopping=True, no_repeat_ngram_size=2)

            loss = model(input_ids=input_ids, labels=labels).loss
            total_loss += loss.item()

            replaced_predictions, replaced_labels = ids_to_words(tokenizer, pred_ids, labels)
            result = compute_metrics(replaced_predictions, replaced_labels)

            all_results.append(result)
            all_predictions.extend(replaced_predictions)
            all_labels.extend(replaced_labels)

    val_loss = total_loss / len(val_loader)
    avg_result = {key: sum(r[key] for r in all_results) / len(all_results) for key in all_results[0]}
    
    writer.add_scalar('Loss/valid', val_loss, epoch)
    writer.add_scalar('ROUGE/rouge-1', avg_result['rouge-1'], epoch)
    writer.add_scalar('ROUGE/rouge-2', avg_result['rouge-2'], epoch)
    writer.add_scalar('ROUGE/rouge-l', avg_result['rouge-l'], epoch)

    return val_loss, avg_result, all_predictions, all_labels

In [14]:
train_step = 0

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
save_path = os.path.join("./T5_runs", timestamp)

weights_path = os.path.join(save_path, 'weights')
logs_path = os.path.join(save_path, 'logs')
os.makedirs(weights_path, exist_ok=True)
os.makedirs(logs_path, exist_ok=True)

best_rouge = 0
early_stopping_counter = 0
best_val_loss = float('inf')
writer = SummaryWriter(log_dir=logs_path)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=T_0, T_mult=T_mult, eta_max=max_lr,  T_up=warmup_epochs, gamma=T_gamma)

for epoch in range(1, epochs + 1):
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch}/{epochs}, Current Learning Rate: {current_lr:.6f}")
    train_loss = train(epoch, model, device, train_loader, optimizer, writer, accumulation_steps)
    val_loss, val_result, val_predictions, val_labels = validate(tokenizer, model, device, val_loader, writer, epoch)

    # avg_rouge = (val_result['rouge-1'] + val_result['rouge-2'] + val_result['rouge-l']) / 3
    print(f"Train Loss: {train_loss:.6f}, Valid Loss: {val_loss:.6f}")
    print(f"Rouge-1: {val_result['rouge-1']:.6f}, Rouge-2: {val_result['rouge-2']:.6f}, Rouge-l: {val_result['rouge-l']:.6f}")

    scheduler.step()
    print('-'*150)
    for i in range(3):
        print(f"PRED: {val_predictions[i].strip()}")
        print(f"GOLD: {val_labels[i]}")
        print('-'*150)

    # if avg_rouge > best_rouge:
    if val_result['rouge-2'] > best_rouge:
        # best_rouge = avg_rouge
        best_rouge = val_result['rouge-2']
        early_stopping_counter = 0 
        torch.save(model.state_dict(), os.path.join(weights_path, 'best.pth'))
        print(f"New best model saved with average ROUGE: {best_rouge:.6f}")

    else:
        early_stopping_counter += 1
        print(f"Not improve. Early stopping counter: {early_stopping_counter}/{patience}")

    if early_stopping_counter >= patience:
        print("Early stopping triggered.")
        break

    torch.save(model.state_dict(), os.path.join(weights_path, f'epoch-{epoch}.pth'))
    print()

writer.close()
torch.save(model.state_dict(), os.path.join(weights_path, 'last.pth'))
print("Training completed. Last model saved.")

Epoch 1/1000, Current Learning Rate: 0.000001


Train Loss: 2.216769, Valid Loss: 1.773889
Rouge-1: 0.134115, Rouge-2: 0.026682, Rouge-l: 0.132053
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 이스트리트는 요즘 숨쉬기가 좀 힘들어요. 감기는 아니에요. 그냥 숨을 쉴 때마다 가슴이 무겁게 느껴져요.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 헬스장에서 만나자. 몇 시에 갈 건지, 오늘은 다리와 팔목을 운동하자.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 요즘에는 더 건강한 음식을 먹기 시작했어.  요즘에 더 건강하게 먹기 시작한 것 같아.


Train Loss: 1.048891, Valid Loss: 1.462695
Rouge-1: 0.139023, Rouge-2: 0.028105, Rouge-l: 0.136067
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 흞 쉴 때마다 가슴이 무겁게 느껴져요.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 헬스장에서 만나자. 몇 시에 갈지, 오늘은 다리와 팔목을 운동하자. 금요일에 다리를 할 수 있어.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 겏단은 더 이상 건강에 해로운 음식을 먹는 것을 멈춰야 한다고 말했고, 림단도 요즘에는 더 건강한 음식을 먹기 시작했다고 말했다

Train Loss: 0.820631, Valid Loss: 1.074047
Rouge-1: 0.150027, Rouge-2: 0.029636, Rouge-l: 0.145844
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 흞어 쉴 때마다 가슴이 무겁게 느껴져요. 숨을 쉴 때 숨이 무겁고, 알러지 증상이 항상 나타나나요? 운동을 할 때 많이 나타나요?
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 이스트리트는 3시 30분에 헬스장에 가자고 제안하고, 이스트는 농구를 했는데 다리가 아프다며 팔과 배를 운동하자고 한다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 이프티스는 더

Train Loss: 0.612231, Valid Loss: 0.833479
Rouge-1: 0.196014, Rouge-2: 0.045840, Rouge-l: 0.186476
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person2# 은 의사 선생님에게 최근 숨쉬기가 힘들다고 말하고, 알레르기가 있는지 묻고, 운동을 할 때 많이 나타난다고 말한다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 이 씨는 지미에게 3시 30분에 헬스장에 가자고 제안한다. 그는 오늘은 다리와 팔목을, 오늘은 팔과 배를 운동하자고 제안하고, 금요일에 다리를 할 수 있다고 말한다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
-----------------------------------------------------------------------------------------------------------------------------------------

...

Train Loss: 0.274756, Valid Loss: 0.532493
Rouge-1: 0.360490, Rouge-2: 0.124021, Rouge-l: 0.339272
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 은 숨쉬기가 힘들다고 말합니다. #Person2# 는 알레르기가 없고 운동을 할 때 많이 나타난다고 합니다. 의사는 #Person1# 을 폐 전문의에게 보내 천식에 대한 검사를 받게 할 것입니다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 지미는 #Person1# 에게 운동하러 가자고 제안하지만, #Person2# 는 다리가 아프다고 말한다. 그들은 금요일에 다리를 하기로 결정한다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
-------------------------------------------------------------------------------------------------------

Train Loss: 0.271659, Valid Loss: 0.531183
Rouge-1: 0.363121, Rouge-2: 0.125410, Rouge-l: 0.341263
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 은 숨쉬기가 힘들다고 말합니다. #Person2# 는 알레르기가 없고 운동을 할 때 많이 나타난다고 합니다. 의사는 #Person1# 을 폐 전문의에게 보내 천식에 대한 검사를 받게 할 것입니다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 지미는 #Person1# 에게 운동하러 가자고 제안하지만, #Person2# 는 다리가 아프다고 말한다. 그들은 금요일에 다리를 하기로 결정한다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
-------------------------------------------------------------------------------------------------------

Train Loss: 0.268923, Valid Loss: 0.532500
Rouge-1: 0.364237, Rouge-2: 0.125024, Rouge-l: 0.341593
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 은 숨쉬기가 힘들다고 말합니다. #Person2# 는 알레르기가 없고 운동을 할 때 많이 나타난다고 합니다. 의사는 #Person1# 을 천식에 대한 검사를 받게 할 것입니다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 지미는 #Person1# 에게 3시 30분에 운동하자고 제안하지만, #Person2# 는 다리가 아프다고 말한다. 그들은 금요일에 다리를 하기로 결정한다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
-------------------------------------------------------------------------------------------------------------

Train Loss: 0.266338, Valid Loss: 0.530589
Rouge-1: 0.365088, Rouge-2: 0.125878, Rouge-l: 0.344162
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 은 숨쉬기가 힘들다고 말합니다. #Person2# 는 알레르기가 없지만 운동을 할 때 많이 나타납니다. 의사는 폐 전문의에게 천식 검사를 받게 할 것입니다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 지미는 #Person1# 에게 3시 30분에 운동하자고 제안한다. 그들은 금요일에 다리를 할 수 있도록 두 날을 바꾸기로 한다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
---------------------------------------------------------------------------------------------------------------------------------------

Train Loss: 0.263811, Valid Loss: 0.531498
Rouge-1: 0.364906, Rouge-2: 0.126435, Rouge-l: 0.342587
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 은 최근 숨쉬기가 힘들다고 말합니다. #Person2# 는 알레르기가 없지만 운동을 할 때 많이 나타난다고 합니다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                                 
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 지미는 #Person1# 에게 3시 30분에 운동하자고 제안하지만, #Person2# 는 다리가 아프다고 말한다. 그들은 금요일에 두 날을 바꾸기로 한다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                         
-----------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
def predict(tokenizer, model, device, test_loader, fname):
    model.eval()
    summary = []
    with torch.no_grad():
        for input_ids in tqdm(test_loader):
            input_ids = input_ids.to(device, dtype=torch.long)

            pred_ids = model.generate(
                input_ids=input_ids,
                max_length=sum_max_len, 
                num_beams=4,
                repetition_penalty=2.0, 
                length_penalty=1.0, 
                early_stopping=True,
                no_repeat_ngram_size=2
            )
            for ids in pred_ids:
                result = tokenizer.decode(ids)
                summary.append(result)
                
    # remove_tokens = ['<usr>', f"{tokenizer.unk_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": fname,
            "summary" : preprocessed_summary,
        }
    )
    return output

In [16]:
model.load_state_dict(torch.load(f'{save_path}/weights/best.pth'))
output = predict(tokenizer, model, device, test_loader, test_df['fname'])
output.to_csv(f"{save_path}/prediction.csv", index=False)

100%|██████████| 125/125 [00:48<00:00,  2.58it/s]
